In [1]:
import numpy as np
import torch
import gym
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
from environment import *
from mix_state_env import MixStateEnv
from config import *
import copy
from MyGlobal import MyGlobals
from itertools import count
from torch.distributions import Categorical
import random
import math
from collections import namedtuple, deque
import pickle

with open('D:/Lab/RL/AODAI_RL/code/dqn_new_statedict2.pkl', 'rb') as file:
    new_model_statedict = pickle.load(file)
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
result_train2=list()
result_test2=list()
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

state_size = NUM_STATE
action_size = NUM_ACTION
lr = 0.001
eps_start=0.9
eps_end=0.05
eps_decay=1000


class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)
        self.Transition = namedtuple('Transition',('state', 'action', 'next_state', 'reward'))
    def push(self, *args):
        self.memory.append(self.Transition(*args))
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    def __len__(self):
        return len(self.memory)
replay = ReplayMemory(10000)
class DQNnet(nn.Module):
    def __init__(self,n_observations,n_actions):
        super(DQNnet,self).__init__()
        self.layer1=nn.Linear(n_observations,128)
        self.layer2=nn.Linear(128,256)
        self.layer3=nn.Linear(256,128)
        self.layer4=nn.Linear(128,n_actions)
    def forward(self,x):
        x=F.relu(self.layer1(x))
        x=F.relu(self.layer2(x))
        x=F.relu(self.layer3(x))
        return self.layer4(x)

class DQNAgent:
    def __init__(self) :
        self.env=MixStateEnv()
        self.optimize=0
        self.env.seed(123)
        self.batch_size=128
        self.eps_start=0.9
        self.eps_end=0.05
        self.eps_decay=1000
        self.tau_start=0.5
        self.tau_end=0.01
        self.tau_decay=1000
        self.tau=0.05
        self.tau2=0.85
        self.lr=1e-4
        self.n_actions=NUM_ACTION
        self.n_observations=NUM_STATE
        self.policy_net=DQNnet(self.n_observations,self.n_actions).to(device)
        self.target_net=DQNnet(self.n_observations,self.n_actions).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        # self.target_net.load_state_dict(new_model_statedict)
        # self.policy_net.load_state_dict(new_model_statedict)
        self.optimizer=optim.AdamW(self.policy_net.parameters(),lr=self.lr,amsgrad=True)
        self.memory=ReplayMemory(10000)
        self.stepdone=0
        self.stepdone1=0
    def select_action(self,state):
            sample=random.random()
            eps_threshold=self.eps_end+(self.eps_start-self.eps_end)*math.exp(-1.*self.stepdone/self.eps_decay)
            self.stepdone+=0.05
            if sample > eps_threshold:
                with torch.no_grad():
                    return self.policy_net(state).max(-1)[1].view(1, 1)
            else:
                return torch.tensor([[random.randint(0,NUM_ACTION-1)]], device=device, dtype=torch.long)    

    def optimize_model(self,gamma):
            # self.start_time=time.time()
            if len(self.memory) < self.batch_size:
                return
            transitions = self.memory.sample(self.batch_size)
            batch = Transition(*zip(*transitions))

            
            non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                                batch.next_state)), device=device, dtype=torch.bool)
            non_final_next_states = torch.cat([s for s in batch.next_state
                                                        if s is not None])
            batch_state=[]
            for i in batch.state :
                batch_state.append(i.view(1,16))
            state_batch = torch.cat(batch_state)
            action_batch = torch.cat(batch.action)

           
            reward_batch = torch.cat(batch.reward)

            
            state_action_values = self.policy_net(state_batch).gather(1, action_batch)

            next_state_values = torch.zeros(self.batch_size, device=device)
            with torch.no_grad():
                 next_state_values[non_final_mask] =self.target_net(non_final_next_states).max(1)[0]
                # next_state_values[non_final_mask] =self.policy_net(non_final_next_states).gather(1,self.target_net(non_final_next_states).max(1)[1].unsqueeze(0))
                
            # Compute the expected Q values
            expected_state_action_values = (next_state_values * gamma) + reward_batch
            

            # Compute Huber loss
            criterion = nn.SmoothL1Loss()
            loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

            # Optimize the model
            self.optimizer.zero_grad()
            loss.backward()
            # In-place gradient clipping
            torch.nn.utils.clip_grad_value_(self.policy_net.parameters(), 100)
            self.optimizer.step()
            self.optimize+=1
            # self.end_time=time.time()
            # print(f"optimze_time:{self.end_time-self.start_time},times{self.optimize}")
    def train(self,num_iters,num_episodes,duration,gamma):
    
            for iter in range(num_iters):
                self.env.replay()
                for episode in range(num_episodes):
                    state = self.env.reset()
                    done = False
                    while not done:
                        for i in count():
                            state = torch.FloatTensor(state).to(device)
                            action = self.select_action(state)
                            action1=action.item()
                            next_state, reward, done, _= self.env.step(np.array(action1))
                            reward = torch.tensor([reward], device=device)
                            if done:
                                if (self.env.old_avg_reward < -1500):
                                    return
                                next_state = None
                                print('Episode: {}, Score: {}'.format(
                                    episode, self.env.old_avg_reward))
                                result_train2.append(self.env.old_avg_reward*-800)

                                break
                            
                                
                            if (i > duration):
                                break
                            next_state = torch.tensor(next_state, dtype=torch.float32, device=device).unsqueeze(0)
                            self.memory.push(state, action, next_state, reward)
                            state = next_state

                            self.optimize_model(gamma)
                            self.tau1=self.tau_end+(self.tau_start-self.tau_end)*math.exp(-1.*self.stepdone1/self.tau_decay)
                            self.stepdone1+=0.005

                            target_net_state_dict = self.target_net.state_dict()
                            policy_net_state_dict = self.policy_net.state_dict()
                        
                            for key in policy_net_state_dict:
                                target_net_state_dict[key] = policy_net_state_dict[key]*self.tau1 + target_net_state_dict[key]*(1-self.tau1)
                            self.target_net.load_state_dict(target_net_state_dict)

    def test(self,num_episodes):
            
            if (self.env.old_avg_reward < -1500):
                return
            for episode in range(num_episodes):
                state = self.env.reset()
                done = False

                while not done:
                    state = torch.FloatTensor(state).to(device)
                    action=action = self.select_action(state)
                    action1=action.item()
                    next_state, reward, done, _= self.env.step(np.array(action1))

                    state = next_state

                print('Test Episode: {}, Score: {}'.format(episode, self.env.old_avg_reward))
                result_test2.append(self.env.old_avg_reward*-800)
    def runAC(self,i, dur,gamma):
    # MyGlobals.folder_name = "Actor_Critic_800_30s/dur" + str(dur) + "/" + str(i) +'/'
        MyGlobals.folder_name = f"test/gamma{gamma}/dur{dur}/{i}/"
        self.train(num_iters=9, num_episodes=121,
            duration=dur,gamma=gamma )
        self.test( num_episodes=60, )

Agent=DQNAgent()



[WinError 183] Cannot create a file when that file already exists: 'd:\\Lab\\RL\\AODAI_RL\\result/result3/Not changing/'


In [2]:
Agent.runAC(1,31,0.99)

[93, 118, 98, 87, 108, 109, 97, 90]
Episode: 0, Score: -0.1100205630596806
[84, 127, 90, 94, 100, 109, 104, 92]
Episode: 1, Score: -0.1216604152106387
[93, 132, 91, 90, 97, 97, 96, 104]
Episode: 2, Score: -0.10512573704278619
[79, 131, 95, 94, 103, 110, 99, 89]
Episode: 3, Score: -0.08300257605856425
[66, 151, 97, 96, 90, 100, 106, 94]
Episode: 4, Score: -0.04640588198502129
[74, 135, 95, 96, 103, 103, 104, 90]
Episode: 5, Score: -0.08543001174100759
[66, 106, 109, 97, 98, 110, 110, 104]
Episode: 6, Score: -0.07397223206365953
[75, 116, 104, 104, 94, 96, 104, 107]
Episode: 7, Score: -0.06626237163519137
[75, 109, 103, 102, 104, 104, 98, 105]
Episode: 8, Score: -0.05262149427447138
[68, 134, 97, 111, 89, 88, 108, 105]
Episode: 9, Score: -0.0593100857729378
[56, 152, 85, 107, 93, 104, 110, 93]
Episode: 10, Score: -0.08445236761788567
[75, 161, 94, 103, 110, 79, 93, 85]
Episode: 11, Score: -0.05420070576983892
[57, 169, 87, 108, 102, 97, 96, 84]
Episode: 12, Score: -0.03880772595220264
[4

In [3]:
with open('dqn_train_notrans3.pkl', 'wb') as file:
    pickle.dump(result_train2, file)
with open('dqn_test_notrans3.pkl', 'wb') as file:
    pickle.dump(result_test2, file)

In [5]:
with open('dqn_train18_statedict.pkl', 'wb') as file:
    pickle.dump(Agent.policy_net.state_dict(), file)

In [6]:
import numpy as np
import torch
import gym
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
from environment import *
from mix_state_env import MixStateEnv
from config import *
import copy
from MyGlobal import MyGlobals
from itertools import count
from torch.distributions import Categorical
import random
import math
from collections import namedtuple, deque
import pickle

with open('D:/Lab/RL/AODAI_RL/code/dqn_new_statedict2.pkl', 'rb') as file:
    new_model_statedict = pickle.load(file)
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
result_train2=list()
result_test2=list()
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

state_size = NUM_STATE
action_size = NUM_ACTION
lr = 0.001
eps_start=0.9
eps_end=0.05
eps_decay=1000


class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)
        self.Transition = namedtuple('Transition',('state', 'action', 'next_state', 'reward'))
    def push(self, *args):
        self.memory.append(self.Transition(*args))
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    def __len__(self):
        return len(self.memory)
replay = ReplayMemory(10000)
class DQNnet(nn.Module):
    def __init__(self,n_observations,n_actions):
        super(DQNnet,self).__init__()
        self.layer1=nn.Linear(n_observations,128)
        self.layer2=nn.Linear(128,128)
        
        self.layer3=nn.Linear(128,n_actions)
    def forward(self,x):
        x=F.relu(self.layer1(x))
        x=F.relu(self.layer2(x))
        return self.layer3(x)

class DQNAgent:
    def __init__(self) :
        self.env=MixStateEnv()
        self.optimize=0
        self.env.seed(123)
        self.batch_size=128
        self.eps_start=0.9
        self.eps_end=0.05
        self.eps_decay=1000
        self.tau_start=0.5
        self.tau_end=0.01
        self.tau_decay=1000
        self.tau1=0.05
        self.tau2=0.85
        self.tau=0.05
        self.lr=1e-4
        self.n_actions=NUM_ACTION
        self.n_observations=NUM_STATE
        self.policy_net=DQNnet(self.n_observations,self.n_actions).to(device)
        self.target_net=DQNnet(self.n_observations,self.n_actions).to(device)
        self.save_net=DQNnet(self.n_observations,self.n_actions).to(device)
        self.target_net.load_state_dict(new_model_statedict)
        self.policy_net.load_state_dict(new_model_statedict)
        self.save_net.load_state_dict(new_model_statedict)
        self.optimizer=optim.AdamW(self.policy_net.parameters(),lr=self.lr,amsgrad=True)
        self.memory=ReplayMemory(10000)
        self.stepdone=0
        self.stepdone1=0
    def select_action(self,state):
            sample=random.random()
            eps_threshold=self.eps_end+(self.eps_start-self.eps_end)*math.exp(-1.*self.stepdone/self.eps_decay)
            self.stepdone+=0.05
            if sample > eps_threshold:
                with torch.no_grad():
                    return self.policy_net(state).max(-1)[1].view(1, 1)
            else:
                return torch.tensor([[random.randint(0,NUM_ACTION-1)]], device=device, dtype=torch.long)    

    def optimize_model(self,gamma):
            # self.start_time=time.time()
            if len(self.memory) < self.batch_size:
                return
            transitions = self.memory.sample(self.batch_size)
            batch = Transition(*zip(*transitions))

            
            non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                                batch.next_state)), device=device, dtype=torch.bool)
            non_final_next_states = torch.cat([s for s in batch.next_state
                                                        if s is not None])
            batch_state=[]
            for i in batch.state :
                batch_state.append(i.view(1,16))
            state_batch = torch.cat(batch_state)
            action_batch = torch.cat(batch.action)

           
            reward_batch = torch.cat(batch.reward)

            
            state_action_values = self.policy_net(state_batch).gather(1, action_batch)

            next_state_values = torch.zeros(self.batch_size, device=device)
            with torch.no_grad():
                 next_state_values[non_final_mask] =self.target_net(non_final_next_states).max(1)[0]
                # next_state_values[non_final_mask] =self.policy_net(non_final_next_states).gather(1,self.target_net(non_final_next_states).max(1)[1].unsqueeze(0))
                
            # Compute the expected Q values
            expected_state_action_values = (next_state_values * gamma) + reward_batch
            

            # Compute Huber loss
            criterion = nn.SmoothL1Loss()
            loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

            # Optimize the model
            self.optimizer.zero_grad()
            loss.backward()
            # In-place gradient clipping
            torch.nn.utils.clip_grad_value_(self.policy_net.parameters(), 100)
            self.optimizer.step()
            self.optimize+=1
            # self.end_time=time.time()
            # print(f"optimze_time:{self.end_time-self.start_time},times{self.optimize}")
    def train(self,num_iters,num_episodes,duration,gamma):
    
            for iter in range(num_iters):
                self.env.replay()
                for episode in range(num_episodes):
                    state = self.env.reset()
                    done = False
                    while not done:
                        for i in count():
                            state = torch.FloatTensor(state).to(device)
                            action = self.select_action(state)
                            action1=action.item()
                            next_state, reward, done, _= self.env.step(np.array(action1))
                            reward = torch.tensor([reward], device=device)
                            if done:
                                if (self.env.old_avg_reward < -1500):
                                    return
                                next_state = None
                                print('Episode: {}, Score: {}'.format(
                                    episode, self.env.old_avg_reward))
                                result_train2.append(self.env.old_avg_reward*-800)

                                break
                            
                                
                            if (i > duration):
                                break
                            next_state = torch.tensor(next_state, dtype=torch.float32, device=device).unsqueeze(0)
                            self.memory.push(state, action, next_state, reward)
                            state = next_state

                            self.optimize_model(gamma)
                            self.tau1=self.tau_end+(self.tau_start-self.tau_end)*math.exp(-1.*self.stepdone1/self.tau_decay)
                            self.stepdone1+=0.005
             
                            
                            target_net_state_dict = self.target_net.state_dict()
                            policy_net_state_dict = self.policy_net.state_dict()
                            save_net_state_dict = self.save_net.state_dict()
                            
                            for key in policy_net_state_dict:
                                target_net_state_dict[key] = policy_net_state_dict[key]*self.tau1 + target_net_state_dict[key]*(1-self.tau1)
                                save_net_state_dict[key] = policy_net_state_dict[key]*self.tau2 + save_net_state_dict[key]*(1-self.tau2)

                            self.target_net.load_state_dict(target_net_state_dict)
                            self.save_net.load_state_dict(save_net_state_dict)
                            self.policy_net.load_state_dict(save_net_state_dict)

                            # target_net_state_dict = self.target_net.state_dict()
                            # policy_net_state_dict = self.policy_net.state_dict()
                        
                            # for key in policy_net_state_dict:
                            #     target_net_state_dict[key] = policy_net_state_dict[key]*self.tau1 + target_net_state_dict[key]*(1-self.tau1)
                            # self.target_net.load_state_dict(target_net_state_dict)

    def test(self,num_episodes):
            
            if (self.env.old_avg_reward < -1500):
                return
            for episode in range(num_episodes):
                state = self.env.reset()
                done = False

                while not done:
                    state = torch.FloatTensor(state).to(device)
                    action=action = self.select_action(state)
                    action1=action.item()
                    next_state, reward, done, _= self.env.step(np.array(action1))

                    state = next_state

                print('Test Episode: {}, Score: {}'.format(episode, self.env.old_avg_reward))
                result_test2.append(self.env.old_avg_reward*-800)
    def runAC(self,i, dur,gamma):
    # MyGlobals.folder_name = "Actor_Critic_800_30s/dur" + str(dur) + "/" + str(i) +'/'
        MyGlobals.folder_name = f"test/gamma{gamma}/dur{dur}/{i}/"
        self.train(num_iters=9, num_episodes=121,
            duration=dur,gamma=gamma )
        self.test( num_episodes=60, )

Agent=DQNAgent()



[WinError 183] Cannot create a file when that file already exists: 'd:\\Lab\\RL\\AODAI_RL\\result/result3/test/gamma0.99/dur31/1/'


RuntimeError: Error(s) in loading state_dict for DQNnet:
	Unexpected key(s) in state_dict: "layer4.weight", "layer4.bias". 
	size mismatch for layer2.weight: copying a param with shape torch.Size([256, 128]) from checkpoint, the shape in current model is torch.Size([128, 128]).
	size mismatch for layer2.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for layer3.weight: copying a param with shape torch.Size([128, 256]) from checkpoint, the shape in current model is torch.Size([8, 128]).
	size mismatch for layer3.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([8]).

In [ ]:
Agent.runAC(1,31,0.99)

[98, 106, 106, 99, 102, 105, 92, 92]
Episode: 0, Score: -0.0910445186177657
[74, 131, 108, 102, 105, 93, 99, 88]
Episode: 1, Score: -0.07814384706284343
[74, 117, 104, 100, 104, 103, 103, 95]
Episode: 2, Score: -0.08149966467443721
[86, 105, 114, 99, 109, 105, 106, 76]
Episode: 3, Score: -0.07920642081906068
[73, 121, 102, 103, 106, 106, 113, 76]
Episode: 4, Score: -0.07379089565633275
[66, 155, 105, 98, 102, 102, 97, 75]
Episode: 5, Score: -0.05782486771783448
[81, 171, 105, 102, 92, 98, 88, 63]
Episode: 6, Score: -0.05219485354133198
[77, 180, 98, 85, 96, 107, 101, 56]
Episode: 7, Score: -0.0552343103168795
[69, 180, 103, 99, 99, 94, 94, 62]
Episode: 8, Score: -0.037695145038343994
[61, 143, 101, 90, 97, 99, 103, 106]
Episode: 9, Score: -0.04275338424155224
[54, 139, 109, 89, 98, 92, 111, 108]
Episode: 10, Score: -0.028247179459175865
[58, 142, 105, 99, 106, 87, 102, 101]
Episode: 11, Score: -0.03639026597307992
[64, 117, 108, 99, 109, 92, 103, 108]
Episode: 12, Score: -0.04444622216

In [ ]:
with open('dqn_train_trans5.pkl', 'wb') as file:
    pickle.dump(result_train2, file)
with open('dqn_test_trans5.pkl', 'wb') as file:
    pickle.dump(result_test2, file)